In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

# Exploração inicial

In [2]:
# Leitura do arquivo
df = pd.read_csv('DelayedFlights.csv')

In [ ]:
# Uma olhada inicial nas colunas e estrutura dos dados
print(df.dtypes)
df.iloc[0]

In [3]:
# Dropando essa coluna de Id do dataframe que não precisa
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
# Analisando a correlação das colunas para ver se existe alguma correlação inesperada entre os dados
correlation_matrix = df.corr()
plt.figure(figsize=(15,12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

# Permutation Importance

In [30]:
# Copiando o df para não alterar o original
feature_importance_df = df.copy()

In [31]:
# Verificando quantos voos foram cancelados
feature_importance_df[feature_importance_df['Cancelled'] == 1].shape

(633, 29)

In [32]:
# Removendo os voos cancelados, poucas observações ok de remover
feature_importance_df = feature_importance_df[feature_importance_df['Cancelled'] != 1]

In [ ]:
# Verificando valores nulos nas colunas
feature_importance_df.isnull().sum()

In [33]:
# Removendo voos com tempos de voo nulos, poucas observações ok de remover
feature_importance_df.dropna(subset = ['AirTime'], inplace = True)

In [15]:
# Verificando valores nulos nas colunas
feature_importance_df.isnull().sum()

Year                      0
Month                     0
DayofMonth                0
DayOfWeek                 0
DepTime                   0
CRSDepTime                0
ArrTime                   0
CRSArrTime                0
UniqueCarrier             0
FlightNum                 0
TailNum                   3
ActualElapsedTime         0
CRSElapsedTime            0
AirTime                   0
ArrDelay                  0
DepDelay                  0
Origin                    0
Dest                      0
Distance                  0
TaxiIn                    0
TaxiOut                   0
Cancelled                 0
CancellationCode          0
Diverted                  0
CarrierDelay         680883
WeatherDelay         680883
NASDelay             680883
SecurityDelay        680883
LateAircraftDelay    680883
dtype: int64

In [34]:
# As colunas que sobraram como nulos podem ser preenchidas com 0 já que de fato os null significam 0 minutos de atrasos daquele tipo.
# Na coluna placa teve 3 nulos, como são só 3 e provavelmente é uma coluna irrelevante para a previsão, ok por 0 nelas
feature_importance_df.fillna({
    'TailNum': 'N',
    'CarrierDelay': 0,
    'WeatherDelay': 0,
    'NASDelay': 0,
    'SecurityDelay': 0,
    'LateAircraftDelay': 0
}, inplace = True)

In [17]:
# Verificando valores nulos nas colunas
feature_importance_df.iloc[0]

Year                   2008
Month                     1
DayofMonth                3
DayOfWeek                 4
DepTime              2003.0
CRSDepTime             1955
ArrTime              2211.0
CRSArrTime             2225
UniqueCarrier            WN
FlightNum               335
TailNum              N712SW
ActualElapsedTime     128.0
CRSElapsedTime        150.0
AirTime               116.0
ArrDelay              -14.0
DepDelay                8.0
Origin                  IAD
Dest                    TPA
Distance                810
TaxiIn                  4.0
TaxiOut                 8.0
Cancelled                 0
CancellationCode          N
Diverted                  0
CarrierDelay            0.0
WeatherDelay            0.0
NASDelay                0.0
SecurityDelay           0.0
LateAircraftDelay       0.0
Name: 0, dtype: object

In [35]:
# Dict para salvar o de para do label encoding
de_para_dict = {}
# Aplicando LabelEncoder em cada coluna
for column in ['UniqueCarrier', 'TailNum', 'Origin', 'Dest', 'CancellationCode']:
    le = LabelEncoder()
    feature_importance_df[column] = le.fit_transform(feature_importance_df[column])
    # Armazenando o De Para para cada coluna
    de_para_dict[column] = dict(zip(le.classes_, range(len(le.classes_))))

In [37]:
# Criando um atributo alvo para ser previsto pelo Randon Forest e em seguida feita a análise de importancia
feature_importance_df['Delayed'] = (df['ArrDelay'] > 15).astype(int)

## Código para calcular o feature importance
Estudar melhor o funcionamento da função permutation_importance e explorar mais os dados dependendo

In [38]:
# Separando features e target
X = feature_importance_df.drop(columns=['Delayed'])
y = feature_importance_df['Delayed']

In [41]:
# Treinando o modelo Random Forest
rf = RandomForestClassifier()
rf.fit(X, y)

RandomForestClassifier()

In [42]:
# Calculando Permutation Importance
result = permutation_importance(rf, X, y, n_repeats=10, random_state=0)

# Criando DataFrame para mostrar as importâncias
permutation_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': result.importances_mean
}).sort_values(by='Importance', ascending=False)

KeyboardInterrupt: 